In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Load dataset

In [ ]:
# Load dataset MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("x_train: {}".format(x_train.shape))
print("y_train: {}".format(y_train.shape))
print("x_test: {}".format(x_test.shape))
print("y_test: {}".format(y_test.shape))

### Explore dataset

In [ ]:
# Plot examples
plt.figure(figsize=(16, 10))
for i in range(10):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(y_train[i])
    plt.axis("off")
plt.show()

In [ ]:
# Plot labels distribution
plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.title("Training set")
ax = sns.distplot(y_train, bins=10, rug=True, kde=False)

plt.subplot(1, 2, 2)
plt.title("Test set")
ax = sns.distplot(y_test, bins=10, rug=True, kde=False)

### Prepare dataset

In [ ]:
# Normalization
x_train = x_train / 255.
x_test = x_test / 255.

print("x_train range: [{}, {}]".format(np.min(x_train), np.max(x_train)))
print("x_test range: [{}, {}]".format(np.min(x_test), np.max(x_test)))

In [ ]:
# Add channel dimension
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

print("x_train shape: {}".format(x_train.shape))
print("x_test shape: {}".format(x_test.shape))

In [ ]:
# Split train/val
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, shuffle=True, stratify=y_train)

print("x_train: {}".format(x_train.shape))
print("y_train: {}".format(y_train.shape))
print("x_val: {}".format(x_val.shape))
print("y_val: {}".format(y_val.shape))

### Build model

In [ ]:
# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, 3, strides=1, padding="same", activation="relu", input_shape=x_train.shape[-3:]),
    tf.keras.layers.Conv2D(8, 3, strides=2, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(16, 3, strides=2, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(32, 3, strides=2, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(64, 3, strides=2, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(128, 3, strides=2, padding="same", activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(10)
])
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["acc"])
model.summary()

### Train model

In [ ]:
# Train model
history = model.fit(x_train, y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_data=[x_val, y_val])

In [ ]:
# PLot training history
loss = history.history["loss"]
val_loss = history.history["val_loss"]
acc = history.history["acc"]
val_acc = history.history["val_acc"]

x = list(range(len(loss)))

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.plot(x, loss, "b", x, val_loss, "r")
plt.title("Loss")
plt.subplot(1, 2, 2)
plt.plot(x, acc, "b", x, val_acc, "r")
plt.title("Accuracy")
plt.show()

### Evaluate

In [ ]:
# Get test set score
loss, acc = model.evaluate(x_test, y_test, verbose=0)

print("Test loss: {}".format(loss))
print("Test accuracy: {}".format(acc))

In [ ]:
# Show prediction result
preds = model.predict(x_train[:10])
y_pred = np.argmax(preds, axis=-1)

plt.figure(figsize=(16, 10))
for i in range(10):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i][:, :, 0], cmap='gray')
    plt.title(y_pred[i])
    plt.axis("off")
plt.show()

In [ ]:
# Show fail prediction
preds = model.predict(x_train)
y_preds = np.argmax(preds, axis=-1)
fail_indices = np.where(y_train != y_preds)[0]
    
plt.figure(figsize=(16, 10))
for i, idx in enumerate(fail_indices[:10]):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[idx][:, :, 0], cmap='gray')
    plt.title("{} != {}".format(y_preds[idx], y_train[idx]))
    plt.axis("off")
plt.show()